In [2]:
import pandas as pd
df = messages = pd.read_csv("SMSSpamCollection", sep='\t',
                            names=['label', 'message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

In [4]:
# independent features
x = list(df['message'])

In [5]:
# dependent features
y = list(df['label'])


In [6]:
y = list(pd.get_dummies(y,drop_first=True)['spam'])

In [7]:
### train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

In [8]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.8 MB/s 
     |████████████████████████████████| 101 kB 15.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
train_encodings = tokenizer(x_train, truncation = True, padding = True)
test_encodings = tokenizer(x_test, truncation = True, padding = True)

## Convert these encodings into Dataset objects

In [11]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [12]:
train_dataset 

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [13]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',               # output directory
    num_train_epochs = 2,                 # total number of training epochs
    per_device_train_batch_size=8,        # batch size per device during training
    per_device_eval_batch_size=16,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='./logs',                 # directory for storing logs
    logging_steps=10,
    eval_steps = 10        # eval_steps = 10; to solve type error '>', nonetype and int
)

In [15]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

  trainer = TFTrainer(
      model = model,        # the instantiated HF transformers model to be trained
      args = training_args,       # training arguments, defined above
      train_dataset = train_dataset,    # training dataset
      eval_dataset = test_dataset        # evaluation dataset
  )

  trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [16]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.022303201471056258}

In [17]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.086318 , -2.8201454],
       [-2.6524317,  3.1842077],
       [ 3.0362592, -2.788354 ],
       ...,
       [ 2.0011132, -1.7090563],
       [-2.6588888,  3.1447616],
       [ 2.9525445, -2.6811674]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.022319517816816056})

In [18]:
trainer.predict(test_dataset)[1].shape

(1115,)

In [20]:
output = trainer.predict(test_dataset)[1]

In [21]:
### confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, output)
cm

array([[955,   0],
       [  0, 160]])

In [22]:
## save
trainer.save_model('senti_model')